# HW4: Parser Combinators (PL 2025 Fall @ HNU CE)
- 이름: 이재석
- 학번: 20210492

----
과제 내용:
- `operator :: Parser Char Tok` 작성하고 적절히 테스트
- `integer :: Parser Char Tok` 작성하고 적절히 테스트
- `lexer :: Parser Char [Tok]` 작성하고 적절히 테스트
- `var :: Parser Tok Expr`     작성하고 적절히 테스트
- `expr :: Parser Tok Expr`    작성하고 적절히 테스트

In [4]:
:opt no-lint
{-# LANGUAGE TypeSynonymInstances FlexibleInstances #-}

In [5]:
type Parser tok a = [tok] -> [(a, [tok])]

In [6]:
return :: a -> Parser tok a -- 입력 토큰열 ts를 소비하지 않고
return v = \ts -> [(v,ts)]  -- 그냥 v를 리턴하며 성공하는 파서

failure :: Parser tok a -- 입력에 관계없이 무조건 실패하는 파서
failure = \_ -> []

In [7]:
:type return True

return True :: forall {tok}. Parser tok Bool

In [8]:
(return True) "abcd"

[(True,"abcd")]

In [9]:
failure "abcd"

[]

In [10]:
item :: Parser tok tok
item []     = []       -- 길이 0인 토큰열에 대해서는 실패
item (t:ts) = [(t,ts)] -- 맨 앞의 토큰 t하나만 처리해 t를 리턴 

In [11]:
item "abcd"
item "ab"
item "a"

[('a',"bcd")]

[('a',"b")]

[('a',"")]

In [12]:
item ""

[]

In [13]:
(>>=) :: Parser tok a -> (a -> Parser tok b) -> Parser tok b
p1 >>= pf = \ts -> [ (v2,ts2) | (v1,ts1) <- p1 ts,
{- 이어붙이기 sequencing -}       (v2,ts2) <- (pf v1) ts1 ]

In [14]:
(item >>= \_ -> item) "abcd"

[('b',"cd")]

In [15]:
(item >>= \c1 ->
 item >>= \c2 ->
 return [c1,c2]  ) "abcd"

[("ab","cd")]

In [16]:
sat :: (tok -> Bool) -> Parser tok tok
sat test = item >>= \t ->          -- 토큰 하나를 읽어들여
           if test t then return t -- 조건에 맞는 경우에만 성공하고 
                     else failure  -- 그렇지 않으면 실패하는 파서

In [17]:
(<|>) :: Parser tok a -> Parser tok a -> Parser tok a  -- 선택 choice
p1 <|> p2 = \ts -> case p1 ts of
                     []  -> p2 ts  -- 첫번째 파서가 실패하면 두번째로
                     rs1 -> rs1    -- 첫번째가 성공하면 첫번째만으로

In [18]:
:type sat (`elem` "123") <|> sat (`elem` "abc")

sat (`elem` "123") <|> sat (`elem` "abc") :: Parser Char Char

In [19]:
'b' `elem` "abc"

True

In [20]:
p_123_or_abc = sat (`elem` "123") <|> sat (`elem` "abc")

p_123_or_abc "456abc"
p_123_or_abc "def456"
p_123_or_abc "234bcd"
p_123_or_abc "bcd234"

[]

[]

[('2',"34bcd")]

[('b',"cd234")]

In [21]:
many, many1  :: Parser tok a -> Parser tok [a]  -- 상호재귀적으로 정의됨
many  p = many1 p <|> return [] -- 1회 성공 또는 무조건 0회라 치고 성공

many1 p = p      >>= \v  ->  -- 한번 성공한 다음
          many p >>= \vs ->  -- 0회 이상 성공
          return (v:vs)

In [22]:
many (sat (`elem` "abc")) "ddaabbcc"
many (sat (`elem` "abc")) "aabbcc" 
many (sat (`elem` "abc")) "aabbddcc" 

[("","ddaabbcc")]

[("aabbcc","")]

[("aabb","ddcc")]

In [23]:
many1 (sat (`elem` "abc")) "ddaabbcc"
many1 (sat (`elem` "abc")) "aabbcc" 
many1 (sat (`elem` "abc")) "aabbddcc" 

[]

[("aabbcc","")]

[("aabb","ddcc")]

# 문자열을 처리하는 파서

In [24]:
import Data.Char ( isDigit, isLower, isUpper,
                   isAlpha, isAlphaNum, isSpace )
digit = sat isDigit
lower = sat isLower
upper = sat isUpper
letter = sat isAlpha
alphanum = sat isAlphaNum
space = sat isSpace

In [25]:
char :: Char -> Parser Char Char    -- char c는 주어진 글자 c와 첫글자가
char c = sat (==c)                  -- 일치하는 경우에만 성공하는 파서

string :: String -> Parser Char String  -- string s는 주어진 문자열 s와
string []     = return []               -- 앞부분이 일치하는 경우에만 성공
string (c:cs) = char c    >>= \_ ->
                string cs >>= \_ ->
                return (c:cs)

In [26]:
nat :: Parser Char Int
nat = many1 digit >>= \s ->  -- 1개 이상의 digit(숫자)
      return (read s)        -- 문자열 s를 정수로 변환한 값으로 성공시킴

ident :: Parser Char String
ident = lower         >>= \c  ->  -- 첫글자는 소문자
        many alphanum >>= \cs ->  -- 그 뒤에는 0개 이상의 문자 또는 숫자
        return (c:cs)

spaces  :: Parser Char ()
spaces  = many  space >>= \_ -> return ()  -- 0개 이상의 공백문자 처리
spaces1 :: Parser Char ()
spaces1 = many1 space >>= \_ -> return ()  -- 1개 이상의 공백문자 처리

# 어휘분석(토큰화)
Lexical Analysis

Lexing

Tokenizing

이항 연산자인 덧셈(`+`), 곱셈(`*`), 거듭제곱(`^`)을 지원하는 변수와 정수 상수를 포함한 수식의 `lexer` 작성

변수의 첫 글자는 알파벳 소문으로 시작하고 나머지 글자는 알파벳 대소문자 또는 숫자로 이루어질 수 있다.

In [27]:
data Tok = OP String -- 연산자
         | ID String -- 변수 이름
         | INT Int   -- 정수
         | LP        -- (
         | RP        -- )
         deriving (Eq,Ord,Show)

In [52]:
-- 변수 이름
varname :: Parser Char Tok
varname = ident  >>= \s -> return (ID s)

-- 자연수
natural :: Parser Char Tok
natural = nat >>= \n -> return (INT n)

In [53]:
varname "ab123 + 123"
varname "123 + ab123"

[(ID "ab123"," + 123")]

[]

In [54]:
natural "ab123 + 123"
natural "123 + ab123"

[]

[(INT 123," + ab123")]

In [55]:
-- 연산자 +, *, ^ 중 하나
operator :: Parser Char Tok
operator = sat (`elem` "+-*/^") >>= \c -> return (OP [c])

In [56]:
-- operator 적절히 테스트
operator "*abc"
operator "-" --실패
operator "^123" 

[(OP "*","abc")]

[(OP "-","")]

[(OP "^","123")]

In [57]:
option :: a -> Parser tok a -> Parser tok a
option x p = p <|> return x

In [63]:
-- 0이나 12와 같은 자연수와 -12같은 음수 형태의 문자열 모두 처리
integer :: Parser Char Tok
integer = (char '-' >> many1 digit >>= \s -> return (INT (negate (read s))))
          <|> (many1 digit >>= \s -> return (INT (read s)))


In [35]:
-- integer 적절히 테스트
test_integer1 = integer "123"
test_integer1

[(INT (-123),"")]

In [37]:
tok :: Parser Char a -> Parser Char a
tok p = p      >>= \v ->
        spaces >>= \_ ->
        return v

In [64]:
lexer :: Parser Char [Tok]
lexer = spaces >> many token
  where
    token = tok operator
         <|> tok (char '(' >> return LP)
         <|> tok (char ')' >> return RP)
         <|> tok integer
         <|> tok varname
    tok p = p >>= \t -> spaces >> return t

In [50]:
-- lexer 적절히 테스트
-- 커널이 죽어버림
test_lexer_simple = lexer "123"

# 문법분석
Syntax 

Parsing

In [40]:
data Expr = Lit Int            -- n
          | Var String         -- x
          | Add Expr Expr      -- e1 + e2  (좌결합, 우선순위 가장 낮음)
          | Mul Expr Expr      -- e1 * e2  (좌결합, 우선순위 중간)
          | Exp Expr Expr      -- e1 ^ e2  (우결합, 우선순외 가장 높음) 
          deriving (Eq, Ord, Show)

In [71]:
isINT :: Tok -> Bool
isINT (INT _) = True
isINT _       = False

int2lit :: Tok -> Expr
int2lit (INT n) = (Lit n)
int2lit t       = error (show t ++ " is not INT")

In [72]:
-- INT 토큰만 Lit로 분석하는 파서
lit :: Parser Tok Expr
lit = sat isINT >>= \t -> return (int2lit t)

In [73]:
lit [INT 3, OP "+", INT 2]
lit [ID "x", OP "+", INT 2]

[(Lit 3,[OP "+",INT 2])]

[]

In [74]:
-- ID 토큰만 Var로 분석하는 파서
var :: Parser Tok Expr
var = sat isID >>= \(ID s) -> return (Var s) -- lit 와 비슷한 방법으로 작성
  where
    isID (ID _) = True
    isID _      = False

In [75]:
var [ID "x", OP "+", INT 1] -- [(Var "x", [OP "+", INT 1])]
var [INT 1] 

[(Var "x",[OP "+",INT 1])]

[]

In [79]:
--모르겠습니다..
expr :: Parser Tok Expr
expr = undefined -- 적절히 작성

-- 힌트: expr0, expr1, expr2, ... 와 같이 결합 단계별 파서로 나누어 작성. 물론 lit와 var도 활용

In [82]:
-- expr0, expr1, expr2, ... 및 expr 파서를 적절히 테스트

In [70]:
-- expr0, expr1, expr2, ... 및 expr 파서를 적절히 테스트

In [49]:
-- expr0, expr1, expr2, ... 및 expr 파서를 적절히 테스트

In [72]:
-- expr0, expr1, expr2, ... 및 expr 파서를 적절히 테스트

In [73]:
-- expr0, expr1, expr2, ... 및 expr 파서를 적절히 테스트

In [74]:
-- expr0, expr1, expr2, ... 및 expr 파서를 적절히 테스트